<span style="color:green;font-size:20pt">German English Translation</span>

In [1]:
!"E:\python\python312\python.exe" -m spacy download de_core_news_sm

     ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
     ------------------------- -------------- 9.2/14.6 MB 51.8 MB/s eta 0:00:01
     ---------------------------------- ---- 13.1/14.6 MB 32.9 MB/s eta 0:00:01
     --------------------------------------- 14.6/14.6 MB 31.7 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [2]:
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
import torchtext
import torch

In [3]:
multi30k.URL["train"] = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/training.tar.gz"
multi30k.URL["valid"] = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/validation.tar.gz"

In [4]:
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

In [5]:
train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
data_set = iter(train_iter)
for n in range(5):
    # Getting the next pair of source and target sentences from the training data set
    src, tgt = next(data_set)

    # Printing the source (German) and target (English) sentences
    print(f"sample {str(n+1)}")
    print(f"Source ({SRC_LANGUAGE}): {src}\nTarget ({TGT_LANGUAGE}): {tgt}")

sample 1
Source (de): Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.
Target (en): Two young, White males are outside near many bushes.
sample 2
Source (de): Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem.
Target (en): Several men in hard hats are operating a giant pulley system.
sample 3
Source (de): Ein kleines Mädchen klettert in ein Spielhaus aus Holz.
Target (en): A little girl climbing into a wooden playhouse.
sample 4
Source (de): Ein Mann in einem blauen Hemd steht auf einer Leiter und putzt ein Fenster.
Target (en): A man in a blue shirt is standing on a ladder cleaning a window.
sample 5
Source (de): Zwei Männer stehen am Herd und bereiten Essen zu.
Target (en): Two men are at the stove preparing food.


In [6]:
german, english = next(data_set)
print(f"Source German ({SRC_LANGUAGE}): {german}\nTarget English  ({TGT_LANGUAGE}): { english }")
token_transform = {}

token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')
token_transform['de'](german)

Source German (de): Ein Mann in grün hält eine Gitarre, während der andere Mann sein Hemd ansieht.
Target English  (en): A man in green holds a guitar while the other man observes his shirt.


['Ein',
 'Mann',
 'in',
 'grün',
 'hält',
 'eine',
 'Gitarre',
 ',',
 'während',
 'der',
 'andere',
 'Mann',
 'sein',
 'Hemd',
 'ansieht',
 '.']

In [7]:
token_transform['en'](english)

['A',
 'man',
 'in',
 'green',
 'holds',
 'a',
 'guitar',
 'while',
 'the',
 'other',
 'man',
 'observes',
 'his',
 'shirt',
 '.']

In [8]:
# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

In [9]:
vocab_transform = {}

In [10]:
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    # Define a mapping to associate the source and target languages
    # with their respective positions in the data samples.
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    # Iterate over each data sample in the provided dataset iterator
    for data_sample in data_iter:
        # Tokenize the data sample corresponding to the specified language
        # and yield the resulting tokens.
        yield token_transform[language](data_sample[language_index[language]])

In [11]:
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Training data iterator
    train_iterator = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    #To decrease the number of padding tokens, you sort data on the source length to batch similar-length sequences together
    sorted_dataset = sorted(train_iterator, key=lambda x: len(x[0].split()))
    # Create torchtext's Vocab object
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(sorted_dataset, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

In [12]:
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)

In [13]:
seq_en=vocab_transform['en'](token_transform['en'](english))
print(f"English text string: {english}\n English sequence: {seq_en}")

seq_de=vocab_transform['de'](token_transform['de'](german))
print(f"German text string: {german}\n German sequence: {seq_de}")


English text string: A man in green holds a guitar while the other man observes his shirt.
 English sequence: [6, 12, 7, 51, 144, 4, 126, 29, 8, 75, 12, 1748, 27, 23, 5]
German text string: Ein Mann in grün hält eine Gitarre, während der andere Mann sein Hemd ansieht.
 German sequence: [5, 12, 7, 657, 39, 18, 133, 8, 37, 16, 105, 12, 136, 41, 1779, 4]


In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [17]:
# function to add BOS/EOS, flip source sentence and create tensor for input sequence indices
def tensor_transform_s(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.flip(torch.tensor(token_ids), dims=(0,)),
                      torch.tensor([EOS_IDX])))

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform_t(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

In [18]:
seq_en=tensor_transform_s(seq_en)
seq_en

tensor([   2,    5,   23,   27, 1748,   12,   75,    8,   29,  126,    4,  144,
          51,    7,   12,    6,    3])

In [19]:
seq_de=tensor_transform_t(seq_de)
seq_de

tensor([   2,    5,   12,    7,  657,   39,   18,  133,    8,   37,   16,  105,
          12,  136,   41, 1779,    4,    3])

In [20]:
# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
text_transform = {}

text_transform[SRC_LANGUAGE] = sequential_transforms(token_transform[SRC_LANGUAGE], #Tokenization
                                            vocab_transform[SRC_LANGUAGE], #Numericalization
                                            tensor_transform_s) # Add BOS/EOS and create tensor

text_transform[TGT_LANGUAGE] = sequential_transforms(token_transform[TGT_LANGUAGE], #Tokenization
                                            vocab_transform[TGT_LANGUAGE], #Numericalization
                                            tensor_transform_t) # Add BOS/EOS and create tensor


In [21]:
# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_sequences = text_transform[SRC_LANGUAGE](src_sample.rstrip("\n"))
        src_sequences = torch.tensor(src_sequences, dtype=torch.int64)
        tgt_sequences = text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n"))
        tgt_sequences = torch.tensor(tgt_sequences, dtype=torch.int64)
        src_batch.append(src_sequences)
        tgt_batch.append(tgt_sequences)

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX,batch_first=True)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX,batch_first=True)
    
    return src_batch.to(device), tgt_batch.to(device)

In [22]:
BATCH_SIZE = 4

train_iterator = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
sorted_train_iterator = sorted(train_iterator, key=lambda x: len(x[0].split()))
train_dataloader = DataLoader(sorted_train_iterator, batch_size=BATCH_SIZE, collate_fn=collate_fn,drop_last=True)

valid_iterator = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
sorted_valid_dataloader = sorted(valid_iterator, key=lambda x: len(x[0].split()))
valid_dataloader = DataLoader(sorted_valid_dataloader, batch_size=BATCH_SIZE, collate_fn=collate_fn,drop_last=True)


src, trg = next(iter(train_dataloader))
src,trg

C:\Users\DEC0D\AppData\Local\Temp\ipykernel_20728\2458598798.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_sequences = torch.tensor(src_sequences, dtype=torch.int64)
C:\Users\DEC0D\AppData\Local\Temp\ipykernel_20728\2458598798.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_sequences = torch.tensor(tgt_sequences, dtype=torch.int64)


(tensor([[    2,     3,     1,     1,     1],
         [    2,  5510,     3,     1,     1],
         [    2,  5510,     3,     1,     1],
         [    2,  1701,     8, 12642,     3]]),
 tensor([[   2,    3,    1,    1,    1,    1,    1,    1,    1,    1,    1],
         [   2, 6650, 4623,  259,  172, 9953,  115,  692, 3428,    5,    3],
         [   2,  216,  110, 3913, 1650, 3823,   71, 2808, 2187,    5,    3],
         [   2,    6, 3398,  202,  109,   37,    3,    1,    1,    1,    1]]))